<a href="https://colab.research.google.com/github/futuremojo/nlp-demystified/blob/main/notebooks/nlpdemystified_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Demystified | Word Vectors
https://nlpdemystified.org<br>
https://github.com/futuremojo/nlp-demystified<br><br>
Course module for this demo: https://www.nlpdemystified.org/course/word-vectors

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

To demonstrate word vectors, we're going to use **Gensim** which we first encountered in the [topic modelling](https://www.nlpdemystified.org/course/topic-modelling) module.<br>

At the time of this recording, the default Gensim version in Colab was 3.X, so we'll first upgrade to 4.X.

In [2]:
# Upgrade gensim just in case.
%pip install -U gensim==4.*

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB 1.3 MB/s eta 0:00:19
   ---------------------------------------- 0.2/24.0 MB 2.6 MB/s eta 0:00:10
    --------------------------------------- 0.5/24.0 MB 4.6 MB/s eta 0:00:06
   - -------------------------------------- 0.9/24.0 MB 5.5 MB/s eta 0:00:05
   - -------------------------------------- 1.1/24.0 MB 5.1 MB/s eta 0:00:05
   -- ------------------------------------- 1.2/24.0 MB 4.9 MB/s eta 0:00:05
   -- ------------------------------------- 1.2/24.0 MB 4.6 MB/s eta 0:00:05
   -- ------------------------------------- 1.2/24.0 MB 4.6 MB/s eta 0:00:05
   -- ------------------------------------- 1.5/24.0 MB 3.9 MB/s eta 0:00:06
   -- ------------------------------------- 1.8/24.0 MB 4.1 MB/s eta 0:00:06
   --- ------------------------------------ 1.9/24.0 MB 4.2 MB/s eta 0:00:06
   --- ------------------------------------ 2.1/24.0 MB 4.2 MB/s eta 0:00:06
   ---

In [1]:
%pip install pandas
%pip install spacy

     ---------------------------------------- 10.7/10.7 MB 3.6 MB/s eta 0:00:00
     -------------------------------------- 502.5/502.5 kB 7.8 MB/s eta 0:00:00
     -------------------------------------- 341.8/341.8 kB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 12.1/12.1 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 181.6/181.6 kB ? eta 0:00:00
     -------------------------------------- 122.2/122.2 kB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB ? eta 0:00:00
     -------------------------------------- 381.6/381.6 kB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 4.5 MB/s eta 0:00:00
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ---------------------------------------- 50.1/50.1 kB 2.7 MB/s eta 0:00:00
     ------------------------------------- 481.9/481.9 kB 10.0 MB/s eta 0:00:00
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
     ---------------------------------------- 1.9/1.9 MB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 45.0/45.0 kB ? eta 0:00:00
Note: you may need to restart the kerne

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install scikit-learn
%pip install tensorflow

     ---------------------------------------- 9.3/9.3 MB 3.7 MB/s eta 0:00:00
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf

from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

**NOTE**<br>
In this notebook, we won't train standalone word embeddings from scratch. Rather, we'll:
1. Use *pretrained* embeddings in one model.
2. Train embeddings alongside another model.
<br>

If you want to try training standalone word embeddings, coding Skip-Gram With Negative Sampling (SGNS) from scratch shouldn't be too hard now that you know all the details. But I recommend just using the **Gensim** library instead:<br>
https://radimrehurek.com/gensim/models/word2vec.html<br>
https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html



# Using Pretrained, Third-Party Vectors

There are a variety of pretrained, static word vector packages out there. In this section, we'll use the **Google News** vectors, a collection of three million, 300-dimension word vectors trained from three billion words from a Google News corpus (circa 2015).

We'll need to first download the actual word vectors. It's over a gigabyte in size but will fit within our environment.

In [26]:

!gdown "https://drive.google.com/uc?id=1BpfbHu4denceXiv8yfdY3EHgjKIcULku"


In [ ]:
embedding_file = './GoogleNews-vectors-negative300.bin.gz'

Next, we'll have **gensim** load the vectors through the **KeyedVectors** module which will enable us to look up vectors by tokens and indices.<br>
https://radimrehurek.com/gensim/models/keyedvectors.html
<br><br>
To save time and space, we'll limit ourselves to 200,000 word vectors for now.

In [ ]:
%%time
word_vectors = KeyedVectors.load_word2vec_format(embedding_file, binary=True, limit=200000)

Retrieving a word's vector is a matter of using a token as a key.

In [ ]:
pizza = word_vectors['pizza']
print(f'Vector dimension: {pizza.shape}')

# The embedding for the word 'pizza'.
print(pizza)

We can get the cosine similarity between two words using the *similarity* method.
https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similarity

In this case, we see words which we expect to be related have a higher similarity measure...

In [ ]:
print(word_vectors.similarity('pizza', 'tomato'))
print(word_vectors.similarity('pizza', 'sauce'))
print(word_vectors.similarity('pizza', 'cheese'))

...compared to words we don't expect to be related.

In [ ]:
print(word_vectors.similarity('pizza', 'gorilla'))
print(word_vectors.similarity('pizza', 'tree'))
print(word_vectors.similarity('pizza', 'yoga'))

Out-of-vocabulary (OOV) words thrown an exception. Bear in mind that looking up even common words here can result in an exception because we loaded only a subset of the vocabulary.

In [ ]:
try:
  word_vectors['womblyboo']
except KeyError as e:
  print(e)

We can compare two sentences by using *n_similarity* which computes the cosine similarity of two **sets** of words.
https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.n_similarity<br><br>
*n_similarity* expects a sentence as a list of words, hence the use of '.split()'. It'll take this list of words, calculate the average of its word vectors, and use the result as an embedding for the whole sentence.

In [ ]:
word_vectors.n_similarity("dog bites man".split(), "canine nips human".split())

In [ ]:
word_vectors.n_similarity("martian dolphins are hostile".split(), "i flunked calligraphy school".split())

One downside of this approach is that word order is thrown out, so two sentences with identical words which mean different things would score a perfect similarity score.

In [ ]:
word_vectors.n_similarity("dog bites man".split(), "man bites dog".split())

Doing quick-and-dirty similarity measures like this is probably best if your corpus is domain-specific and similarity is based more on keywords. The more specific, the better.<br><br>
For example, a corpus of business news headlines would probably work well.

In [ ]:
s1 = "Volkswagen intends to double electric car sales in China".lower().split()
s2 = "First Toyota with solid state battery will be hybrid".lower().split()
word_vectors.n_similarity(s1, s2)

The *most_similar* method returns the words with the closest vectors.<br>
https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar

In [ ]:
word_vectors.most_similar(positive=['cell'], topn=10)

We can also combine vectors first, and retrieve the words most similar to their mean. Here, we're combining the vectors for 'cell' and 'phone' and retrieving the vectors closest to that result.

In [ ]:
word_vectors.most_similar(positive=['cell', 'phone'], topn=10)

Given a collection of words, the *doesn't_match* method returns the word that "doesn't go" with the rest (i.e. with the vector that's furthest away from the mean of all the other vectors).<br>
https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.doesnt_match

In [ ]:
word_vectors.doesnt_match(["apple", "orange", "hamburger", "banana", "kiwi"])

We can see the power of context in this example with 'Toyota' being correctly identified as the odd one out.

In [ ]:
word_vectors.doesnt_match(["Microsoft", "Apple", "Toyota", "Amazon", "Netflix", "Google"])

Visualizing word vectors is straight-forward and can offer insights into what kind of contexts the training algorithm picked up.<br><br>
Because these word vectors have a dimension of 300, we need to reduce them down to two dimensions to plot them on a regular graph. This can be done through **Principal Components Analysis (PCA)**:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html<br>
<br>
Here, we're plotting the words we considered in the slides.

In [ ]:
def display_pca_scatterplot(model, words):        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(10,10))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r', s=128)
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
display_pca_scatterplot(word_vectors, ['swim', 'swimming', 'cat', 'dog', 'feline', 'road', 'car', 'bus'])

We can even solve analogies (to a limited extent) with vector arithmetic.<br><br>
Here, we're solving the analogy:<br>
_Rome is to Italy as London is to __________.<br><br>
Arithmetically, this is Italy + London - Rome.


In [ ]:
word_vectors.most_similar(positive=['Italy', 'London'], negative=['Rome'], topn=3)

Visualizing it can help with geometric intuition.

In [ ]:
display_pca_scatterplot(word_vectors, ['Rome', 'Italy', 'London', 'Britain', 'UK'])

# Using Pretrained Word Vectors for Classification

In this section, we'll train a **Keras** model to use these Google News vectors to perform sentiment analysis on a bunch of **Yelp** reviews.
<br><br>
For this model, we'll increase the number of word vectors loaded to 1,000,000.



In [ ]:
%%time
word_vectors = KeyedVectors.load_word2vec_format(embedding_file, binary=True, limit=1000000)

The dataset we'll use is *Yelp Polarity Reviews*, a collection of ~600,000 reviews for both training and testing.<br><br>
The original Yelp reviews use a five-star rating system. The ratings in this dataset have been modified to simply be negative (label==1) or positive (label==2).<br>
https://www.tensorflow.org/datasets/catalog/yelp_polarity_reviews<br><br>
Tensorflow comes with a datasets loader but we're going to download the file manually and process the data ourselves for completeness.

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz"

Unzipping the archive results in *train.csv* and *test.csv* files placed in the default *contents* folder of our environment.

In [ ]:
!tar xvzf /root/input/yelp_review_polarity_csv.tgz

# Show current working directory.
!pwd

The **Pandas** library makes it simple to load a CSV file into memory and manipulate the data.<br>
https://pandas.pydata.org/<br>
https://pandas.pydata.org/docs/<br>
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html?highlight=read_csv

Here, we're loading the CSV into a Pandas **dataframe** (sort of like an in-memory table) and explicitly naming the columns.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas.DataFrame

In [ ]:
yelp_train = pd.read_csv('yelp_review_polarity_csv/train.csv', names=['sentiment', 'review'])
print(yelp_train.shape)

We can get a quick view of the data through the *head* method.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html

In [ ]:
yelp_train.head()

To save on training time, we'll train on 100,000 reviews rather than the full set. To do that, we'll shuffle the dataset using the *sample* method and *copy* the first 100,000 entries. The reason to shuffle first is to ensure we get a mix of reviews from a variety of businesses (in case the data is sorted in some way).<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html


In [ ]:
TRAIN_SIZE = 100000
yelp_train = yelp_train.sample(frac=1, random_state=1)[:TRAIN_SIZE].copy()
print(yelp_train.shape)

The next thing to do is adjust the labels. This is a **binary classification problem**, so our model's output layer will be a single unit with a **sigmoid** activation function. This function's output will be between 0 and 1 which is then compared against the training label. But the labels are currently 1 for negative, and 2 for positive, which is going to cause problems when calculating the loss.<br><br>
So we'll simply replace the 1s with 0s, and 2s with 1s using the *replace* method.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
<br><br>
Alternatively, we could keep the labels as-is and treat this as a **multiclassification** problem with two labels and use a **softmax**, but we would then need to **one-hot encode** the labels (at least based on what we've learnt so far).


In [ ]:
yelp_train['sentiment'].replace(to_replace=1, value=0, inplace=True)
yelp_train['sentiment'].replace(to_replace=2, value=1, inplace=True)

In [ ]:
yelp_train.head() 

As we've done throughout this course, we'll create train/validation splits.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
yelp_train_split, yelp_val_split = train_test_split(yelp_train, train_size=0.85, random_state=1)

In [ ]:
# Set up training data.
train_reviews = yelp_train_split['review']
y_train = np.array(yelp_train_split['sentiment'])

# Set up validation data.
val_reviews = yelp_val_split['review']
y_val = np.array(yelp_val_split['sentiment'])

A quick sanity check to see how our data is distributed (e.g. balanced or skewed).

In [ ]:
collections.Counter(y_train)

Because we're relying more on richer encodings (in this case, word vectors), we won't perform as much preprocessing this time around. We'll stick with using the regular Keras **tokenizer** and just filter out numbers and certain symbols.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer<br><br>
We'll also have the tokenizer limit itself to tokenizing only the most frequent 20,000 words. This way, the model will focus on the most frequent descriptive sentiment words.

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=20000,
                                               filters='0123456789!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                                               lower=True)

The active selection is creating a `Tokenizer` object from the `keras.preprocessing.text` module. This `Tokenizer` is a crucial part of text preprocessing in Natural Language Processing (NLP) tasks. It helps to convert the text data into numerical data, which can be fed into machine learning models.

Here's a breakdown of the parameters used in the `Tokenizer`:

- `num_words=20000`: This parameter sets the maximum number of words to be kept based on the frequency of words. In this case, only the top 20000 most frequent words will be considered, and less frequent words will be discarded.

- `filters='0123456789!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'`: This string contains all the characters that will be removed from the texts during the tokenization process. Here, it includes digits, punctuation, and special characters. This is useful for simplifying the text data and focusing on the words.

- `lower=True`: This parameter, when set to True, converts all the text into lowercase. This is done to ensure that the same words in different cases are not considered as different words. For example, 'Hello' and 'hello' will be treated as the same word.

After this `Tokenizer` is created, it can be used to fit on the text data and transform it into sequences of integers with the `fit_on_texts` and `texts_to_sequences` methods, respectively.

Build the vocabulary.

In [ ]:
%%time
tokenizer.fit_on_texts(train_reviews)

The active selection is using the `fit_on_texts` method of the `Tokenizer` object to fit the tokenizer on the training reviews. Here's a breakdown:

- `%%time`: This is a magic command in Jupyter notebooks that times the execution of a specific cell.

- `tokenizer.fit_on_texts(train_reviews)`: This line is doing the main work.

  - `fit_on_texts`: This is a method of the `Tokenizer` class that updates the internal vocabulary based on a list of texts. This method should be used before `texts_to_sequences` or `texts_to_matrix` to ensure the tokenizer has been fitted on the texts.

  - `train_reviews`: This is the list of texts that the tokenizer is being fitted on. In this case, it's the training reviews.

After running this cell, the tokenizer will have been fitted on the training reviews, and you can use it to convert these reviews (or other texts) to sequences or matrices.

Next steps:
- Convert the training reviews to sequences using `tokenizer.texts_to_sequences(train_reviews)`.
- Check the word index of the tokenizer using `tokenizer.word_index` to see the mapping of words to integers.
- Try fitting the tokenizer on different texts and see how it affects the word index and the sequences/matrices it generates.

The next step is to vectorize our reviews. In the [_Neural Network Foundations_](https://github.com/futuremojo/nlp-demystified/blob/main/notebooks/nlpdemystified_neural_networks_foundations.ipynb) notebook, we used the *texts_to_matrix* method to turn text into binary bags of words.<br><br>
Here, we're going to use the *text_to_sequences* method to turn each review into a sequence of integers, with each integer representing its corresponding token.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences



In [1]:
%%time
X_train = tokenizer.texts_to_sequences(train_reviews)

NameError: name 'tokenizer' is not defined

In [ ]:
# The first review in the training set, vectorized.
print(X_train[0])

We can look up the corresponding tokens using the tokenizer's *index_word* dict. Here are the tokens corresponding to the first three integers from the first vectorized review.

In [ ]:
[tokenizer.index_word[x] for x in X_train[0][:3]]

We can also convert the integer sequence back to text using the *sequences_to_texts* method, and compare it against the original text.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#sequences_to_texts

In [ ]:
# Review excerpt reconstructed from integer sequence.
tokenizer.sequences_to_texts([X_train[0]])[0][:300]

In [ ]:
# Original review text.
train_reviews.iloc[0][:300]

Some models and situations require us to **pad** our sequences to the same length. While that's not the case here, it can still be beneficial to have all our inputs (and consequently, our batches) to be of uniform size to help with optimizations.<br><br>
In this case, we'll make all our reviews 200 tokens in length (in practice, you can choose a number based on some analysis). So the reviews longer than 200 tokens will be truncated, while the reviews shorter than 200 will be padded with zeroes.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
MAX_REVIEW_LEN = 200
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=MAX_REVIEW_LEN)

In [ ]:
print(X_train[0])
print(X_train[1])

Our training set is prepared. We can now also vectorize and pad our validation set.

In [ ]:
X_val = tokenizer.texts_to_sequences(val_reviews)
X_val = keras.preprocessing.sequence.pad_sequences(X_val, maxlen=MAX_REVIEW_LEN)

Now we need to incorporate the Google News vectors (currently loaded into gensim) into our Keras model. What we'll do is create an embedding matrix that maps each tokenizer integer to its respective word vector.<br><br>
For example, here's the index for the word "good" from the Keras tokenizer and the word vector for "good" from gensim. We want a matrix which maps the index to the vector.


In [ ]:
print(tokenizer.word_index['good'])

In [ ]:
# Part of the vector for the word 'good'.
print(word_vectors['good'][:50])

We'll create this embedding matrix by first initializing a matrix of zeros, then looping over every word in the tokenizer vocabulary and:
1. Checking if the word has a corresponding vector in gensim.
2. If it does, then copy the vector into the matrix row corresponding to the word's index.

In [ ]:
# + 1 to account for padding token.
num_tokens = len(tokenizer.word_index) + 1

# Initialize a matrix of zeroes of size: vocabulary x embedding dimension.
embedding_dim = 300
embedding_matrix = np.zeros((num_tokens, embedding_dim))

for word, i in tokenizer.word_index.items():
  if word_vectors.has_index_for(word):
    embedding_matrix[i] = word_vectors[word].copy()


In [ ]:
# Quick visual check.
print(embedding_matrix[tokenizer.word_index['good']][:50])

We're ready to build our first model using pretrained word vectors. The first layer we'll add is a Keras **embedding** layer which is essentially a trainable lookup table/matrix.<br>
https://keras.io/api/layers/base_layer/#layer-class<br>
https://keras.io/api/layers/core_layers/embedding/<br><br>
In this case, we'll populate the **embedding** layer with the embedding matrix we created, and set *trainable* to True. This means we'll allow the learning algorithm training the classification model to adjust/fine-tune the word vectors as needed for greater performance. This corresponds to one of the scenarios we covered in the slides.

In [ ]:
embedding_layer = layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    input_length=MAX_REVIEW_LEN,
    trainable=True
)

We'll use a simple architecture for this model. Each training example is a sequence of *integers* which gets converted to a sequence of *vectors* (embeddings), but subsequent layers are expecting one vector per review. So we're inserting a **GlobalAveragePooling1D** layer after the embedding layer to average out all the word vectors into a single vector, before sending it further into the network. For classification, this can be pretty effective as a base model approach.<br>
https://keras.io/api/layers/pooling_layers/global_average_pooling1d/<br>

There was no science behind choosing 128 units in the first hidden layer and 64 units in the second hidden layer. The intuition was that the signal would be distilled from 300 dimensions down to 128 dimensions, then down to 64 dimensions before going to output.



In [ ]:
tf.random.set_seed(0)

model = keras.Sequential()

# This layer will output a sequence of 300-dimension *vectors*, one for each element in the input sequence.
model.add(embedding_layer)

# This layer will calculate an average of those vectors.
model.add(layers.GlobalAveragePooling1D())

model.add(layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))
model.add(layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))
model.add(layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Here's an example of what's going to happen under the hood to turn review text into a single vector for the dense layers:

In [ ]:
review = "fantastic papaya steak"
print(f"Review: {review}")

review_sequence = tokenizer.texts_to_sequences([review])
print(f"Review as sequence of integers: {review_sequence}")

review_embeddings = embedding_layer(np.array(review_sequence))
print(f"Review embeddings shape: (Batch size: {review_embeddings.shape[0]}, \
Sequence length: {review_embeddings.shape[1]}, \
Embedding size: {review_embeddings.shape[2]})")

# How our document will be presented to the rest of the neural network.
print(f"Average of embeddings (shape): {np.mean(review_embeddings, axis=1).shape}")

When we call the model's *summary* method, note how there are no params for the **GlobalAveragePooling1D** layer.

In [ ]:
model.summary()

We won't use **early stopping** for this run. This way, we'll be able to compare metrics between the train and validation sets.

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_val, y_val))

In [ ]:
def plot_train_vs_val_performance(history):
  training_losses = history.history['loss']
  validation_losses = history.history['val_loss']

  training_accuracy = history.history['accuracy']
  validation_accuracy = history.history['val_accuracy']

  epochs = range(1, len(training_losses) + 1)

  import matplotlib.pyplot as plt
  fig, (ax1, ax2) = plt.subplots(2)
  fig.set_figheight(15)
  fig.set_figwidth(15)
  fig.tight_layout(pad=5.0)

  # Plot training vs. validation loss.
  ax1.plot(epochs, training_losses, 'bo', label='Training Loss')
  ax1.plot(epochs, validation_losses, 'b', label='Validation Loss')
  ax1.title.set_text('Training vs. Validation Loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Loss')
  ax1.legend()

  # PLot training vs. validation accuracy.
  ax2.plot(epochs, training_accuracy, 'bo', label='Training Accuracy')
  ax2.plot(epochs, validation_accuracy, 'b', label='Validation Accuracy')
  ax2.title.set_text('Training vs. Validation Accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.legend()

  plt.show()

In [ ]:
plot_train_vs_val_performance(history)

We'll initialize a new embedding layer and model and train for epochs equalling the point where we saw the validation loss diverge from the training loss.<br>

**NOTE**: We need to initialize a new embedding layer here because we set the *learnable* parameter to **True** in the previous embedding layer. This means the previous embeddings were almost certainly updated by the learning algorithm. So we're re-training a new model now with the original embeddings.

In [ ]:
embedding_layer = layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    input_length=MAX_REVIEW_LEN,
    trainable=True
)

model = keras.Sequential()
model.add(embedding_layer)
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=512, validation_data=(X_val, y_val))

Now that we have a trained model, let's try it on the test data. As we did with the training data, we'll:
1. Replace the labels with 0 for negative sentiment, and 1 for positive.
2. Convert the reviews into a sequence of integers and pad/truncate each review to a fixed length.

In [ ]:
yelp_test = pd.read_csv('yelp_review_polarity_csv/test.csv', names=['sentiment', 'review'])

In [ ]:
yelp_test['sentiment'].replace(to_replace=1, value=0, inplace=True)
yelp_test['sentiment'].replace(to_replace=2, value=1, inplace=True)
yelp_test.head()

In [ ]:
y_test = np.array(yelp_test['sentiment'])
print(y_test)

In [ ]:
X_test = tokenizer.texts_to_sequences(yelp_test['review'])

In [ ]:
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_REVIEW_LEN)

In [ ]:
model.evaluate(X_test, y_test)

Not bad for a conceptually simple model where we average out a review's word vectors, run it through a few plain hidden layers, and out through a sigmoid function with no regularization and just using defaults for model components (e.g. optimizer settings).<br><br>
We can now use the model for predictions.

In [ ]:
def sentiment(reviews):
  seqs = tokenizer.texts_to_sequences(reviews)
  seqs = keras.preprocessing.sequence.pad_sequences(seqs, maxlen=MAX_REVIEW_LEN)
  return model.predict(seqs)


In [ ]:
# Real reviews from Google Reviews.
pos_review = "The best seafood joint in East Village San Diego!  Great lobster roll, great fish, great oysters, great bread, great cocktails, and such amazing service.  The atmosphere is top notch and the location is so much fun being located just a block away from Petco Park (San Diego Padres Stadium)."
neg_review = "A thoroughly disappointing experience. When you book a Marriott you expect a certain standard. Albany falls way short. Room cleaning has to be booked 24 hours in advance but nobody thought to mention this at check in. The hotel is tired and needs a face-lift. The only bright light in a sea of mediocrity were the pancakes at breakfast. Sadly they weren't enough to save the experience. If you travel to Albany, then do yourself a big favour and book the Westin."

In [ ]:
print(sentiment([pos_review, neg_review]))

# Training New Embeddings and a Model at the Same Time

For this last model, rather than using pretrained embeddings, we'll start with a **random** embedding matrix and let the model come up with its own vectors simultaneously while fitting the training data.<br><br>
We'll also use **early stopping**, but otherwise keep everything else the same.

In [ ]:
tf.random.set_seed(0)

model = keras.Sequential()

# The 'trainable' property is True by default.
model.add(layers.Embedding(input_dim=num_tokens, 
                           output_dim=embedding_dim, 
                           input_length=MAX_REVIEW_LEN))


model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))
model.add(layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))
model.add(layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_val, y_val), callbacks=[es_callback])

In [ ]:
model.evaluate(X_test, y_test)

It looks like in this case, we get comparable performance between fine-tuning pretrained vectors and training embeddings from scratch as part of the model; likely because of the nature of the data and amount of it.

# Try This

In our first model, we used pretrained vectors in the **embedding layer** and set the *trainable* property to **True**, allowing the model to fine-tune the word vectors.<br><br>
Instantiate the same model but this time, set the *trainable* property in the **embedding layer** to **False**. What happens to training performance? Does the training speed increase or decrease? What happens if you try to add some regularization like dropout?<br>

Other things you can try to see what happens: reduce the number of units, use a slower learning rate, reduce the number of hidden layers, reduce vocabulary, reduce embedding dimensions, use regularization, use a shorter sequence length.

In [ ]:
# Instantiate the embedding layer.


model = keras.Sequential()

# Add layers.


# Compile model.


# Call fit.


# Evaluate the model.


# Alternative Static Embedding Algorithms

## GloVe
**GloVe (Global Vectors for Word Representation)** is another algorithm for creating static word vectors. You can read the original GloVe paper and download pretrained word vectors here:<br>
https://nlp.stanford.edu/projects/glove/

## Doc2Vec
An algorithm which represents a document as a dense vector which addresses weaknesses of bag-of-words models.<br>
https://arxiv.org/abs/1405.4053<br>
https://radimrehurek.com/gensim/models/doc2vec.html<br>

## fastText
An alternative approach to creating embeddings. Instead of assigning a vector to each _word_ (e.g. a separate vector each for "dog" and "dogs"), a vector is assigned to each _subword_. For fastText, a subword is defined as a character n-gram.
<br><br>
So if n=3, then a word like "hello" would result in vectors for "<he", "hel", "ell", "llo", "lo>" (note that "<" and ">" are special characters). The vector for "hello" would be the sum of all the above vectors. This helps deal with OOV situations because vectors can still be assigned to unseen words as long as the n-grams exist in the vocabulary.<br>
https://fasttext.cc/<br>
https://radimrehurek.com/gensim/models/fasttext.html
<br><br>
**We'll cover subword tokenization in greater detail later in the course.**